# Import

In [79]:
import pandas as pd
import numpy as np
import sqlite3
import requests
import json
from tqdm.notebook import tqdm

# Read in data and API key

In [37]:
k = open('../data/api_keys.json')
keys = json.load(k)
api_key = keys['google_maps']

In [87]:
hosp = pd.read_csv('../data/neo4j/full_refs_hosp.csv')

In [35]:
hosp['address_provider'].nunique()

2725

In [36]:
hosp['address_hospital'].nunique()

39

# Geocode using Google Geocode API

## Geocode provider addresses

In [67]:
prov_addresses = hosp['address_provider'].unique()

In [68]:
geocode_dict = {}

for address in prov_addresses:
    endpoint = 'https://maps.googleapis.com/maps/api/geocode/json'
    params = {'address': address,
             'key': api_key}

    response = requests.get(endpoint, params)
    geocode_dict[address] = response.json()['results'][0]['geometry']['location']

In [75]:
provider_address_geocodes = pd.DataFrame(geocode_dict).transpose().reset_index().rename(columns = {'index':'address_provider'})

## Geocode hospital addresses

In [78]:
hosp_addresses = hosp['address_hospital'].unique()

In [80]:
geocode_dict_hosp = {}

for address in tqdm(hosp_addresses):
    endpoint = 'https://maps.googleapis.com/maps/api/geocode/json'
    params = {'address': address,
             'key': api_key}

    response = requests.get(endpoint, params)
    geocode_dict_hosp[address] = response.json()['results'][0]['geometry']['location']

  0%|          | 0/39 [00:00<?, ?it/s]

In [83]:
hospital_address_geocodes = pd.DataFrame(geocode_dict_hosp).transpose().reset_index().rename(columns = {'index':'address_hospital'})
hospital_address_geocodes

,address_hospital,lat,lng
0,"313 N MAIN ST, ASHLAND CITY, TN 37015",36.277115,-87.065626
1,"158 HOSPITAL DR, CARTHAGE, TN 37030",36.263555,-85.949974
2,"111 HIGHWAY 70 E, DICKSON, TN 37055",36.082873,-87.337358
3,"555 HARTSVILLE PIKE, GALLATIN, TN 37066",36.390778,-86.432507
4,"4321 CAROTHERS PARKWAY, FRANKLIN, TN 37067",35.916450,-86.820426
5,"2014 QUAIL HOLLOW CIR, FRANKLIN, TN 37067",35.914241,-86.815679
6,"1000 PHYSICIANS WAY, FRANKLIN, TN 37067",35.919140,-86.819249
7,"500 CHURCH ST, HARTSVILLE, TN 37074",36.387772,-86.168102
8,"355 NEW SHACKLE ISLAND RD, HENDERSONVILLE, TN ...",36.325391,-86.622419
9,"5655 FRIST BLVD, HERMITAGE, TN 37076",36.176152,-86.608062


# Write to CSV
So we don't accidentally use up this Google API credit

In [86]:
hospital_address_geocodes.to_csv('../data/hospital_address_geocodes.csv', index = False)
provider_address_geocodes.to_csv('../data/provider_address_geocodes.csv', index = False)